<a href="https://colab.research.google.com/github/dhia-eddine-jedidi/AlFouladhInternship/blob/main/KHATT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iraqyomar/khatt-arabic-hand-written-lines")

print("Path to dataset files:", path)

100%|██████████| 295M/295M [00:03<00:00, 98.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/iraqyomar/khatt-arabic-hand-written-lines/versions/1


In [ ]:
!pip install -q evaluate
!pip install -q datasets jiwer
!pip install -q transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os

# Path to the folders
images_folder = '/root/.cache/kagglehub/datasets/iraqyomar/khatt-arabic-hand-written-lines/versions/1/images'
labels_folder = '/root/.cache/kagglehub/datasets/iraqyomar/khatt-arabic-hand-written-lines/versions/1/labels'
# List to store image paths and labels
data = []

# Iterate over files in images folder
for image_file in os.listdir(images_folder):
    if image_file.endswith(('.png', '.jpg', '.jpeg')):
        # Get image path
        image_path = os.path.join(images_folder, image_file)

        # Get corresponding label
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(labels_folder, label_file)
        if os.path.exists(label_path):
            with open(label_path, 'r', encoding='windows-1256') as f:
                label = f.read().strip()
        else:
            label = None

        # Add image path and label to data list
        data.append({'image_path': image_path, 'label': label})

# Create DataFrame from the data list
df = pd.DataFrame(data)
df.head()

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

import torch
from torch.utils.data import Dataset
from PIL import Image

class Dataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=47):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df['image_path'][idx]
        text = self.df['label'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text,
                                          padding="max_length",
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

from transformers import TrOCRProcessor
from transformers import AutoTokenizer
from transformers import VisionEncoderDecoderModel

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

tokenizer2 = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
processor.tokenizer = tokenizer2

#model defining
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-stage1")
train_dataset = Dataset(root_dir='',
                           df=train_df,
                           processor=processor)
eval_dataset = Dataset(root_dir='',
                           df=test_df,
                           processor=processor)

print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

encoding = train_dataset[0]
for k,v in encoding.items():
  print(k, v.shape)

image = Image.open(train_dataset.root_dir + train_df['image_path'][0]).convert("RGB")
image

labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.tokenizer.decode(labels, skip_special_tokens=True)
print(label_str)
print(labels)

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=16)

import torch.nn as nn
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

#model.config.vocab_size = model.config.decoder.vocab_size

model.decoder.config.vocab_size = processor.tokenizer.vocab_size
model.config.vocab_size = model.config.decoder.vocab_size
model.decoder.output_projection = nn.Linear(256, processor.tokenizer.vocab_size)
model.decoder.model.decoder.embed_tokens = nn.Embedding(processor.tokenizer.vocab_size, 256, padding_idx=1)


# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 47
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 8

# Remplacement de l'importation de load_metric
import evaluate  # Nouvelle API pour les métriques

# Charger les métriques CER et WER
cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")

# Fonctions pour calculer CER et WER (inchangées, mais compatibles avec evaluate)
def compute_cer(pred_ids, label_ids):
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    return cer

def compute_wer(pred_ids, label_ids):
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return wer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

from torch.optim import AdamW
from tqdm.notebook import tqdm
from torch.optim.lr_scheduler import LambdaLR

train_loss_list = []
CER_list = []
WER_list = []
optimizer = AdamW(model.parameters(), lr=2e-5)
print("Learning rate =", optimizer.param_groups[0]['lr'])


for epoch in range(20):  # loop over the dataset multiple times
    # train
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_dataloader):
        # get the inputs
        for k,v in batch.items():
            batch[k] = v.to(device)

        # forward + backward + optimize
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()

    train_loss_list.append(train_loss/len(train_dataloader))
    print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))
    print("Learning rate =", optimizer.param_groups[0]['lr'])

    # evaluate
    model.eval()
    valid_cer = 0.0
    valid_wer = 0.0
    with torch.no_grad():
        for batch in tqdm(eval_dataloader):
            # run batch generation
            outputs = model.generate(batch["pixel_values"].to(device))
            # compute metrics
            cer = compute_cer(pred_ids=outputs, label_ids=batch["labels"])
            wer = compute_wer(pred_ids=outputs, label_ids=batch["labels"])
            valid_cer += cer
            valid_wer += wer
    CER_list.append(valid_cer / len(eval_dataloader))
    WER_list.append(valid_wer / len(eval_dataloader))
    print("Validation CER:", valid_cer / len(eval_dataloader))
    print("Validation WER:", valid_wer / len(eval_dataloader))

model.save_pretrained(".")

import matplotlib.pyplot as plt

# Assuming loss and CER contain values for each epoch
epochs = range(1, len(train_loss_list) + 1)

plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.plot(epochs, train_loss_list, 'b', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(epochs, CER_list, 'r', label='CER')
plt.title('Character Error Rate (CER)')
plt.xlabel('Epochs')
plt.ylabel('CER')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(epochs, WER_list, 'r', label='WER')
plt.title('Word Error Rate (WER)')
plt.xlabel('Epochs')
plt.ylabel('WER')
plt.legend()

plt.tight_layout()
plt.show()


import torch
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import pandas as pd
import os


# Define the display function
def display_image_with_text(image_path, label, text, size=(300, 2000)):
    image = plt.imread(image_path)
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size),
        transforms.ToTensor()
    ])
    image_tensor = transform(image)

    image_np = image_tensor.permute(1, 2, 0).cpu().numpy()
    plt.imshow(image_np)
    print("Label: "+label)
    print("Generated: "+text)
    plt.axis('off')
    plt.show()


# Generate predictions for the test set
for index, row in test_df.head(20).iterrows():
    # Load the image
    image_path = row['image_path']
    original_label = row['label']

    # Load the image as a tensor
    image = plt.imread(image_path)
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((384,384)),
        transforms.ToTensor()
    ])
    image_tensor = transform(image).unsqueeze(0).to(device)

    # Generate prediction
    outputs = model.generate(image_tensor)
    predicted_text = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Display the image with original and predicted text
    display_image_with_text(image_path, original_label, predicted_text)

output_dir = "./finetuned_trocr_arabic"
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)
print(f"Modèle et processor sauvegardés dans {output_dir}")

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-stage1 and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of training examples: 9100
Number of validation examples: 2275
pixel_values torch.Size([3, 384, 384])
labels torch.Size([47])
وتقليل التلون البيئي الناتج عنما والتقليل من إستهلاك الوقود الإحفوري كمصدر للطاقة.
tensor([   33, 13333, 27962,   196,  1152, 19318, 57392,  2464, 11154, 25965,
          322,   323, 25843, 27962,   196,   290, 35889, 25598,  5538,  4404,
         6866,   425,   990,  4444,   162,  6895, 27529,    48,    34,    31,
           31,    31,    31,    31,    31,    31,    31,    31,    31,    31,
           31,    31,    31,    31,    31,    31,    31])


Learning rate = 2e-05


  0%|          | 0/569 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


...

Loss after epoch 2: 4.168063169623511
Learning rate = 2e-05


  0%|          | 0/143 [00:00<?, ?it/s]

Validation CER: 0.7405060637864773
Validation WER: 0.9765389928780085


  0%|          | 0/569 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import torch
import matplotlib.pyplot as plt
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, GenerationConfig
import warnings
import evaluate

# Suppress PIL warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned model and processor
model_dir = "./finetuned_trocr_arabic"
try:
    processor = TrOCRProcessor.from_pretrained(model_dir)
    model = VisionEncoderDecoderModel.from_pretrained(model_dir)
except Exception as e:
    print(f"Error loading model or processor: {e}")
    exit()
model.to(device)
model.eval()

# Set generation config to suppress warning
model.generation_config = GenerationConfig(
    max_length=47,
    num_beams=8,
    early_stopping=True,
    no_repeat_ngram_size=3,
    length_penalty=2.0,
    eos_token_id=processor.tokenizer.sep_token_id
)

# Path to the image
image_path = "/content/Example-of-an-Arabic-Document.png"  # Update if needed
ground_truth = None  # Set to "your_correct_text_here" if known, e.g., "المسند ومن المؤلفات"

# Or use an image from test_df (if available)
# image_path = test_df['image_path'].iloc[0]
# ground_truth = test_df['label'].iloc[0]

# Load and preprocess the image
try:
    image = Image.open(image_path).convert("RGB")  # Ensure RGB format
except Exception as e:
    print(f"Error loading image: {e}")
    exit()

# Preprocess with TrOCRProcessor (resizes to 384x384, RGB)
pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
print(f"Input tensor shape: {pixel_values.shape}")  # Should be [1, 3, 384, 384]

# Generate OCR prediction
with torch.no_grad():
    outputs = model.generate(pixel_values)
predicted_text = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Compute CER/WER if ground truth is available
if ground_truth:
    cer_metric = evaluate.load("cer")
    wer_metric = evaluate.load("wer")
    cer = cer_metric.compute(predictions=[predicted_text], references=[ground_truth])
    wer = wer_metric.compute(predictions=[predicted_text], references=[ground_truth])
    print(f"CER: {cer:.4f}, WER: {wer:.4f}")

# Display the image and predicted text
plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.title(f"Predicted Text: {predicted_text}", fontsize=12)
plt.axis('off')
plt.show()
print(f"Predicted Text: {predicted_text}")
if ground_truth:
    print(f"Ground Truth: {ground_truth}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:34 for open-end generation.


Error loading model or processor: Can't load image processor for './finetuned_trocr_arabic'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './finetuned_trocr_arabic' is the correct path to a directory containing a preprocessor_config.json file
Input tensor shape: torch.Size([1, 3, 384, 384])


ValueError: `decoder_start_token_id` or `bos_token_id` has to be defined for encoder-decoder generation.

In [ ]:
git remote add origin https://github.com/dhia-eddine-jedidi/AlFouladhInternship.git
git branch -M main
git push -u origin main

SyntaxError: invalid syntax (ipython-input-1522874168.py, line 1)